In [17]:
# https://docs.binance.us/?python#get-candlestick-data
# binance.us
# Github.com - send Daniel Github username
# https://algotrading101.com/learn/binance-python-api-guide/
# https://data.binance.vision/
# https://github.com/sammchardy/python-binance

"""
Max requests per minute: 1200 (then IP ban)
Python-binance purposefully implements delays to prevent IP ban due to overloading
Some code is asyncronous
"""

from config import key, secret_key
from binance import Client, ThreadedWebsocketManager, ThreadedDepthCacheManager
import numpy as np
import pandas as pd
import requests
from binance.enums import *
from binance.exceptions import BinanceAPIException, BinanceOrderException

client = Client(key, secret_key, tld='us')

resp = requests.get('https://api.binance.us/api/v3/ping')

print(resp.json())

{}


In [18]:
import requests

resp = requests.get('https://api.binance.us/api/v3/klines?symbol=BTCUSD&interval=1m&limit=1000')

# print(pd.DataFrame(resp.json(), columns = ['open_time','open','high','low','close','volume','close_time','quote_asset_volume','number_of_trades','taker_buy_base_asset','taker_buy_quote_asset_volume', 'ignore']))

In [3]:
# print(client.get_asset_balance(asset='BTC'))
# print(client.get_asset_balance(asset='USD'))
# get market depth
# depth = client.get_order_book(symbol='BTCUSD')

# place a test market buy order, to place an actual order use the create_order function
# order = client.create_test_order(
#     symbol='BNBBTC',
#     side=Client.SIDE_BUY,
#     type=Client.ORDER_TYPE_MARKET,
#     quantity=100)

In [19]:
# get latest price from Binance API
btc_price = client.get_symbol_ticker(symbol="BTCUSDT")
eth_price = client.get_symbol_ticker(symbol="ETHUSDT")
# print full output (dictionary)
print(btc_price)
print(eth_price)


{'symbol': 'BTCUSDT', 'price': '43664.77000000'}
{'symbol': 'ETHUSDT', 'price': '3226.93000000'}


In [20]:
from time import sleep
from datetime import datetime, timedelta
import time
from binance import ThreadedWebsocketManager

btc_price = {'error':False}
def btc_trade_history(msg):
    ''' define how to process incoming WebSocket messages '''
    if msg['e'] != 'error':
        print(msg['c'])
        btc_price['last'] = msg['c']
        btc_price['bid'] = msg['b']
        btc_price['last'] = msg['a']
        btc_price['error'] = False
    else:
        btc_price['error'] = True

In [6]:
# bsm = ThreadedWebsocketManager()
# bsm.start()
# bsm.start_symbol_ticker_socket(callback=btc_trade_history, symbol='BTCUSDT')


In [7]:
# bsm.stop()

In [27]:
# arn:aws:rds:us-east-1:395975088873:db:algomodule
from mysql.connector import connect
connect.raise_on_warnings = False
host = 'algo-1.cqkzwypggftm.us-east-1.rds.amazonaws.com'
port = 3306
username = 'admin'
password = 'Makecashmoney'

connection = connect(host=host,
        user=username,
        password=password)
cursor = connection.cursor()


In [22]:
# import sqlalchemy as db
# engine = db.create_engine(f'mysql://{username}:{password}@{host}:{port}/algomodule')
# engine
# print(cursor.execute("""select * from AlgoModule.eth_hourly"""))


In [28]:
def to_unixmillis(from_date):
    from_date_obj = datetime.strptime(from_date, fmt)
    past = datetime(1970, 1, 1, tzinfo=from_date_obj.tzinfo)
    return int((from_date_obj - past).total_seconds() * 1000.0)


def to_datetime(ms):
    return datetime.fromtimestamp(int(float(ms) / 1000.0))

In [29]:
# valid intervals - 1m, 3m, 5m, 15m, 30m, 1h, 2h, 4h, 6h, 8h, 12h, 1d, 3d, 1w, 1M

# get timestamp of earliest date data is available
start_date = client._get_earliest_valid_timestamp('BTCUSDT', '1d')
start_date = to_datetime(start_date)
fmt = "%Y-%m-%d %H:%M:%S"  # e.g. 2019-11-16 23:16:15
# end_date = time.strftime(fmt, time.localtime())
end_date = datetime.now()


print(start_date)
print(start_date+ timedelta(hours = 12))
print(end_date)


2019-09-22 20:00:00
2019-09-23 08:00:00
2022-04-08 09:25:55.669599


In [25]:
# cursor.execute("""create table AlgoModule.btc_hourly
# (
# open_time datetime,
# close_time datetime,
# open_price double(20,9),
# close_price double(20,9),
# high_price double(20,9), 
# low_price double(20,9),
# volume double(20,9),
# quote_av double(20,9),
# trades int,
# tb_base_av double(20,9),
# tb_quote_av double(20,9)
# )""")
# connection.commit()

In [30]:
def binance_data_to_sql(start_date, end_date, currencytype, timespan, connection):
    cursor = connection.cursor()
    try:
        cursor.execute(f"""create table AlgoModule.{currencytype}_{timespan}
        (
        open_time datetime,
        close_time datetime,
        open_price double,
        close_price double,
        high_price double, 
        low_price double,
        volume double,
        quote_av double,
        trades int,
        tb_base_av double,
        tb_quote_av double
        )""")
        connection.commit()
    except:
        print("Error but moving on")

    step_date = start_date
    while step_date <= end_date:
        print(f'pulling {step_date} data for AlgoModule.{currencytype}_{timespan}')
        bars = client.get_historical_klines(currencytype, timespan, start_str = f'{step_date}', end_str=f'{step_date + timedelta(hours=12)}', limit=1000)
        bars = pd.DataFrame(bars, columns = ['date', 'open',
                    'high', 'low', 'close', 'volume', 'close_time', 'quote_av',
                    'trades', 'tb_base_av', 'tb_quote_av', 'ignore'])
        bars['date'] = bars['date'].apply(to_datetime)
        bars['close_time'] = bars['close_time'].apply(to_datetime)
        for index, row in bars.iterrows():
            cursor.execute(f"""INSERT INTO AlgoModule.{currencytype}_{timespan} (open_time, close_time, open_price, close_price, high_price, low_price,
            volume, quote_av, trades, tb_base_av, tb_quote_av) values(%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s)"""
            , (row.date, row.close_time, row.open, row.close, row.high, row.low, row.volume, row.quote_av, row.trades, row.tb_base_av, row.tb_quote_av))
        step_date = step_date + timedelta(hours=12)
        connection.commit()
    sleep(1)

currency_pairings = ['BTCUSD'
# , 'ETHUSD'
]
time_intervals = [
    # '1m', '3m', '5m', '15m', '30m', '1h', '2h', '4h', '6h',
     '8h', '12h']

for cur in currency_pairings:
    for t in time_intervals:
        binance_data_to_sql(start_date, end_date, cur, t, connection)


pulling 2019-09-22 20:00:00 data for AlgoModule.BTCUSD_8h
pulling 2019-09-23 08:00:00 data for AlgoModule.BTCUSD_8h
pulling 2019-09-23 20:00:00 data for AlgoModule.BTCUSD_8h
pulling 2019-09-24 08:00:00 data for AlgoModule.BTCUSD_8h
pulling 2019-09-24 20:00:00 data for AlgoModule.BTCUSD_8h
pulling 2019-09-25 08:00:00 data for AlgoModule.BTCUSD_8h
pulling 2019-09-25 20:00:00 data for AlgoModule.BTCUSD_8h
pulling 2019-09-26 08:00:00 data for AlgoModule.BTCUSD_8h
pulling 2019-09-26 20:00:00 data for AlgoModule.BTCUSD_8h
pulling 2019-09-27 08:00:00 data for AlgoModule.BTCUSD_8h
pulling 2019-09-27 20:00:00 data for AlgoModule.BTCUSD_8h
pulling 2019-09-28 08:00:00 data for AlgoModule.BTCUSD_8h
pulling 2019-09-28 20:00:00 data for AlgoModule.BTCUSD_8h
pulling 2019-09-29 08:00:00 data for AlgoModule.BTCUSD_8h
pulling 2019-09-29 20:00:00 data for AlgoModule.BTCUSD_8h
pulling 2019-09-30 08:00:00 data for AlgoModule.BTCUSD_8h
pulling 2019-09-30 20:00:00 data for AlgoModule.BTCUSD_8h
pulling 2019-1

KeyboardInterrupt: 

In [61]:
bars

,date,open,high,low,close,volume,close_time,quote_av,trades,tb_base_av,tb_quote_av,ignore
0,2020-04-09 16:00:00,7241.49000000,7307.64000000,7241.49000000,7307.64000000,1.41841300,2020-04-09 16:59:59,10311.62141034,32,0.70216400,5114.03226836,0
1,2020-04-09 17:00:00,7290.69000000,7307.74000000,7282.37000000,7307.74000000,2.91838300,2020-04-09 17:59:59,21290.66035706,28,2.21439400,16157.33611037,0
2,2020-04-09 18:00:00,7307.74000000,7307.74000000,7289.41000000,7292.14000000,5.01766900,2020-04-09 18:59:59,36602.20382031,96,1.04502700,7626.78920645,0
3,2020-04-09 19:00:00,7300.98000000,7300.98000000,7281.54000000,7283.68000000,1.01699700,2020-04-09 19:59:59,7415.97947110,16,0.14680900,1069.98269534,0
4,2020-04-09 20:00:00,7280.13000000,7288.12000000,7250.23000000,7285.68000000,1.68345500,2020-04-09 20:59:59,12234.22997768,39,1.48361200,10782.20263954,0
5,2020-04-09 21:00:00,7288.54000000,7288.54000000,7253.46000000,7263.54000000,0.98019900,2020-04-09 21:59:59,7116.79769244,22,0.23331300,1694.65392574,0
6,2020-04-09 22:00:00,7255.30000000,7265.41000000,7201.00000000,7233.42000000,3.88792800,2020-04-09 22:59:59,28160.01778645,87,1.49005100,10795.24425964,0
7,2020-04-09 23:00:00,7244.71000000,7247.04000000,7165.11000000,7175.02000000,1.58389200,2020-04-09 23:59:59,11405.99817934,60,1.29912500,9359.34487138,0
8,2020-04-10 00:00:00,7155.20000000,7156.41000000,7098.45000000,7143.06000000,9.42647800,2020-04-10 00:59:59,67218.74003297,159,7.27365500,51887.07593257,0
9,2020-04-10 01:00:00,7140.46000000,7140.46000000,6900.00000000,6967.89000000,15.81693800,2020-04-10 01:59:59,110583.26889374,330,7.87945300,55039.47829538,0


In [54]:
connection.commit()

In [8]:
"""
BNB Coin allows for reduced commission fees. 25% cheaper
"""

def topup_bnb(min_balance: float, topup: float):
	''' Top up BNB balance if it drops below minimum specified balance '''
	bnb_balance = client.get_asset_balance(asset='BNB')
	bnb_balance = float(bnb_balance['free'])
	if bnb_balance < min_balance:
		qty = round(topup - bnb_balance, 5)
		print(qty)
		order = client.order_market_buy(symbol='BNBUSDT', quantity=qty)
		return order
	return False

In [9]:
"""
Author: pratikpv
https://github.com/pratikpv/cryptocurrency_data_downloader/blob/master/download_data_from_binance.py
"""

import pandas as pd
import os
import time
from binance.client import Client
from datetime import timedelta, datetime
import pytz
import numpy as np

epoch = 0
fmt = "%Y-%m-%d %H:%M:%S"  # e.g. 2019-11-16 23:16:15
org_columns = ['open',
               'high', 'low', 'close', 'volume', 'close_time', 'quote_av',
               'trades', 'tb_base_av', 'tb_quote_av', 'ignore']

columns_of_interest = ['open', 'high', 'low', 'close', 'volume']


def init_mod():
    # refer: https://www.binance.com/en/support/articles/360002502072 for API keys
    binance_client = Client(api_key=key, api_secret=secret_key, tld='us')
    global epoch
    epoch = datetime.utcfromtimestamp(0)
    return binance_client
def convert_time_to_utc(pst_time):
    utc = pytz.utc
    est = pytz.timezone('US/Eastern')
    datetime1 = datetime.strptime(est_time, fmt)
    est_time = est.localize(datetime1)
    return est_time.astimezone(utc).strftime(fmt)

In [10]:
init_mod()
epoch


datetime.datetime(1970, 1, 1, 0, 0)

In [12]:
def convert_time_to_est(utc_time):
    datetime_obj = datetime.strptime(utc_time, fmt)
    return datetime_obj.replace(tzinfo=time.timezone('UTC')).strftime(fmt)



def download_data_from_binance(symbol, from_date, to_date, output_filename, step=0, pause=-1, simulate=False):
    """
    :param symbol:
    :param from_date:
    :param to_date:
    :param output_filename:
    :param step: step in number of days. Download data in batches of days given by 'step'
    :param pause: pause seconds before downloading next batch.
        if pause == -1 --> random sleep(2,5)
        if pause == 0 --> no sleep
        if pause == num--> sleep for num of seconds
    :param simulate:
    :return:
    """
    binance_client = init_mod()
    from_date_obj = datetime.strptime(from_date, fmt)
    step_date_obj = from_date_obj + timedelta(days=step)
    step_date = step_date_obj.strftime(fmt)

    from_millis = to_unixmillis(from_date)
    to_millis = to_unixmillis(to_date)
    step_millis = to_unixmillis(step_date)

    count = 0
    while True:
        from_millis_str = str(from_millis)
        step_millis_str = str(step_millis)
        print('Step %d:Downloading data from %s to %s' % (count,
                                                          str(to_datetime(from_millis_str)),
                                                          str(to_datetime(step_millis_str))
                                                          ))
        if not simulate:
            # download data

            klines = binance_client.get_historical_klines(symbol, Client.KLINE_INTERVAL_1HOUR,
                                                          from_millis_str, end_str=step_millis_str)
            klines_len = len(klines)
            if klines_len == 0:
                print('\t Failed to download from %s to %s. Got %d' % (str(to_datetime(from_millis_str)),
                                                                       str(to_datetime(step_millis_str)), klines_len
                                                                       ))
                time.sleep(5)

            print('\t Downloaded data of len %d from %s to %s' % (klines_len,
                                                                  str(to_datetime(from_millis_str)),
                                                                  str(to_datetime(step_millis_str))
                                                                  ))
            new_columns = [item + '_' + symbol for item in org_columns]
            new_columns.insert(0, 'timestamp')

            data_df = pd.DataFrame(klines,
                                   columns=new_columns)
            data_df['timestamp'] = pd.to_datetime(data_df['timestamp'], unit='ms')
            data_df.set_index('timestamp', inplace=True)
            data_df.to_csv(output_filename)

        # move to next step of batches
        from_millis = step_millis
        step_date_obj = step_date_obj + timedelta(days=step)
        step_date = step_date_obj.strftime(fmt)
        step_millis = to_unixmillis(step_date)
        count = count + 1
        if pause == -1:
            pause = np.random.randint(2, 5)
        time.sleep(pause)
        if step_millis >= to_millis:
            break


def concat_binance_data(symbol_list, output_filename):
    df_list = []
    for num, symbol in enumerate(symbol_list):
        filename = str('%s-binance-data.csv' % (symbol))
        df = pd.read_csv(filename, index_col=0)
        df_list.append(df)

    result = pd.concat(df_list, axis=1, sort=True)
    result.index = pd.to_datetime(df.index)
    result = result.sort_index().drop_duplicates(keep='first')
    idx = np.unique(result.index, return_index=True)[1]
    result = result.iloc[idx]

    new_columns = [item + '_' + 'BTCUSDT' for item in columns_of_interest]
    # new_columns.insert(0, 'timestamp')

    for num, symbol in enumerate(symbol_list):
        if symbol == 'BTCUSDT':
            continue
        new_columns.append('close_' + symbol)
        new_columns.append('volume_' + symbol)

    result = result[new_columns]
    result.to_csv(output_filename)


def remove_dup_by_index(output_filename):
    result = pd.read_csv(output_filename, index_col=0)
    result.index = pd.to_datetime(result.index)
    result = result.sort_index()        #.drop_duplicates(keep='first')
    idx = np.unique(result.index, return_index=True)[1]
    result = result.iloc[idx]
    result.to_csv(output_filename)


def append_binance_data(master_output_filename, concat_output_filename):
    master_df = pd.read_csv(master_output_filename)
    new_df = pd.read_csv(concat_output_filename)
    master_df = master_df.append(new_df)
    master_df.set_index('timestamp', inplace=True)
    master_df.index = pd.to_datetime(master_df.index)
    master_df = master_df.sort_index().drop_duplicates(keep='first')
    master_df.to_csv(master_output_filename)


# if __name__ == '__main__':
from_date = '2019-11-16 00:00:00'
to_date = time.strftime(fmt, time.localtime())
symbol_list = [
 'BTCUSDT'
#  , 'ETHUSDT'
 ]

for num, symbol in enumerate(symbol_list):
        output_filename = '%s-binance-data.csv' % (symbol)
        print('-' * 60)
        print('Downloading data from %s to %s for %s' % (from_date, to_date, symbol))
        print('-' * 60)
        download_data_from_binance(symbol, from_date, to_date, output_filename, step=1, pause=1, simulate=True)

# concat all currency data
concat_output_filename = 'binance_crypto_data_final_cleaned.csv'
concat_binance_data(symbol_list, concat_output_filename)

# Append the results to master currency data
master_output_filename = 'crypto_data_master_cleaned.csv'
append_binance_data(master_output_filename, concat_output_filename)
remove_dup_by_index(master_output_filename)

------------------------------------------------------------
------------------------------------------------------------
Step 0:Downloading data from 2019-11-15 19:00:00 to 2019-11-16 19:00:00
Step 1:Downloading data from 2019-11-16 19:00:00 to 2019-11-17 19:00:00
Step 2:Downloading data from 2019-11-17 19:00:00 to 2019-11-18 19:00:00
Step 3:Downloading data from 2019-11-18 19:00:00 to 2019-11-19 19:00:00
Step 4:Downloading data from 2019-11-19 19:00:00 to 2019-11-20 19:00:00
Step 5:Downloading data from 2019-11-20 19:00:00 to 2019-11-21 19:00:00
Step 6:Downloading data from 2019-11-21 19:00:00 to 2019-11-22 19:00:00
Step 7:Downloading data from 2019-11-22 19:00:00 to 2019-11-23 19:00:00
Step 8:Downloading data from 2019-11-23 19:00:00 to 2019-11-24 19:00:00
Step 9:Downloading data from 2019-11-24 19:00:00 to 2019-11-25 19:00:00
Step 10:Downloading data from 2019-11-25 19:00:00 to 2019-11-26 19:00:00
Step 11:Downloading data from 2019-11-26 19:00:00 to 2019-11-27 19:00:00
Step 12:Down

KeyboardInterrupt: 